# Project Proposal - Predicting the Likelihood of Heart Conditions based on _________ 

#### Lisa Li, Jovan Kannitho, Justin An, Tammy Ou

## Introduction

Heart conditions can be determiend by a variaety of factors, which include an individual's inherant characteristics, such as age and sex, as well as lifestyle choises, such ask smoking and amount of exercise. The objective of the project is to determine the presence of heart condition based on a person's biological factors. We will analyze Heart Disease Data Set which consists of fourteen variables to measure the 
the health conditions of _____ individuals in Switzerland. Ultimately, our goal is to address: **Does an individual likely to develop heart disease based on their ____ _____ and _____?**

## Preliminary data exploration analysis

In [3]:
library(tidyverse)
library(tidymodels)
library(dplyr)
library(RColorBrewer)
library(kknn)
library(psych)
library(ggcorrplot)

ERROR: Error in library(psych): there is no package called ‘psych’


In [25]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.va.data"
column_names <- c("age",
                  "sex",
                  "chest_pain",
                  "resting_bp",
                  "cholestrol", 
                  "fasting_bp",
                  "resting_ecg",
                  "max_hrate_achieved",
                  "exercise_induced_angina",
                  "st_depression",
                  "slope",
                  "major_vessels",
                  "thalassemia", # too much iron in blood
                  "num")
va_heart_data <- read_csv(url,
                    col_names = c(column_names),
                    col_types = list("d", "f", "f", "d", "d", "f", "f", "d", "f", "d", "f", "i", "f", "d"))


count_sex <- va_heart_data |>
   group_by(sex) |>
   summarize(count = n()) 

                    

va_heart_data_clean <- va_heart_data |>
    mutate(diagnosis = num > 0)    # changed diagnosis levels of 0-4 to True and False
                                   # if True -> presence of heart disease, False -> absence of heart disease

va_heart_data_clean[va_heart_data_clean == "?"] <- NA #changing all ? values to NA
va_heart_data_clean

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”


sex,count
<fct>,<int>
1,194
0,6


In [5]:
#Changing all ? values to NA

va_split <- initial_split(va_heart_data_clean, prop = .75, strata = diagnosis)
va_train <- training(va_split)
va_test <- testing(va_split)

va_train

age,sex,chest_pain,resting_bp,cholestrol,fasting_bp,resting_ecg,max_hrate_achieved,exercise_induced_angina,st_depression,slope,major_vessels,thalassemia,num,diagnosis
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<fct>,<int>,<fct>,<dbl>,<lgl>
44,1,4,130,209,0,1,127,0,0.0,NA,NA,NA,0,FALSE
66,1,3,110,213,1,2,99,1,1.3,2,NA,NA,0,FALSE
66,1,3,120,0,0,1,120,0,-0.5,1,NA,NA,0,FALSE
60,1,3,180,0,0,1,140,1,1.5,2,NA,NA,0,FALSE
56,1,2,126,166,0,1,140,0,0.0,NA,NA,NA,0,FALSE
62,1,4,120,220,0,1,86,0,0.0,NA,NA,NA,0,FALSE
60,1,4,152,0,0,1,118,1,0.0,NA,NA,7,0,FALSE
60,1,4,120,0,0,0,133,1,2.0,1,NA,7,0,FALSE
42,1,3,134,240,NA,0,160,0,0.0,NA,NA,NA,0,FALSE


In [18]:
# va_train |>
# summarize(across(age:diagnosis, mean, na.rm = TRUE))

na_count <- colSums(is.na(va_train))

na_count

#from this na count, we can see that major vessels and thalassemia columns have a lot of missing values, so we will not use these variables as predictors

va_train_select <- va_train |>
                   select(-thalassemia,-major_vessels)


age                     sex              chest_pain 
                      0                       0                       0 
             resting_bp              cholestrol              fasting_bp 
                     42                       4                       6 
            resting_ecg      max_hrate_achieved exercise_induced_angina 
                      0                      40                      40 
          st_depression                   slope           major_vessels 
                     42                      73                     148 
            thalassemia                     num               diagnosis 
                    122                       0                       0

In [20]:
#split our va_train_select into two dataframes: one with our numeric (num) variables, and one with our categorical (cat) variables


va_train_select_num <- va_train_select |>
                        select(age, resting_bp, cholestrol, max_hrate_achieved, st_depression)

va_train_select_cat <- va_train_select |>
                        select(sex, chest_pain, fasting_bp, resting_ecg, exercise_induced_angina, slope)

In [23]:
#calculate the means for all our numeric predictors

means <- va_train_select_num |>
         map_df(mean, na.rm = TRUE)

means



age,resting_bp,cholestrol,max_hrate_achieved,st_depression
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
59.04698,131.7103,173.7862,121.4862,1.316822


In [42]:
do a count for all our categorical variables

count_sex <- va_train_select_cat |>
   group_by(sex) |>
   summarize(count=n()) 

count_sex   # from this count, we can see that basically most of the dataset is of males. 
            # Thus, we should change our analysis scope to only predicitng heart disease for MALES

count_cp <- va_train_select_cat |>
   group_by(chest_pain) |>
   summarize(count=n()) 

count_cp

count_fasting_bp <- va_train_select_cat |>
   group_by(fasting_bp) |>
   summarize(count=n()) 

count_fasting_bp

count_r.ecg <- va_train_select_cat |>
   group_by(resting_ecg) |>
   summarize(count=n()) 

count_r.ecg






ERROR: Error in parse(text = x, srcfile = src): <text>:1:4: unexpected symbol
1: do a
       ^


hihi 